In [1]:
import pandas as pd
from category_encoders.ordinal import OrdinalEncoder

In [2]:
sales_train = pd.read_csv("../input/sales_train_validation.csv")
weights_validation = pd.read_csv("../input/weights_validation.csv")

***

In [3]:
hierarchy = (sales_train.loc[:, ["item_id", "dept_id", "cat_id", "store_id", "state_id"]]
             .drop_duplicates())

In [4]:
item_encoder = OrdinalEncoder()
item_encoder.fit(hierarchy.loc[:, ["item_id"]])
hierarchy.loc[:, "item_id"]  = item_encoder.transform(hierarchy.loc[:, ["item_id"]])

dept_encoder = OrdinalEncoder()
dept_encoder.fit(hierarchy.loc[:, ["dept_id"]])
hierarchy.loc[:, "dept_id"]  = dept_encoder.transform(hierarchy.loc[:, ["dept_id"]])

cat_encoder = OrdinalEncoder()
cat_encoder.fit(hierarchy.loc[:, ["cat_id"]])
hierarchy.loc[:, "cat_id"]   = cat_encoder.transform(hierarchy.loc[:, ["cat_id"]])

store_encoder = OrdinalEncoder()
store_encoder.fit(hierarchy.loc[:, ["store_id"]])
hierarchy.loc[:, "store_id"] = store_encoder.transform(hierarchy.loc[:, ["store_id"]])

state_encoder = OrdinalEncoder()
state_encoder.fit(hierarchy.loc[:, ["state_id"]])
hierarchy.loc[:, "state_id"] = state_encoder.transform(hierarchy.loc[:, ["state_id"]])

***
### weights for level 12

In [7]:
weights_level12 = (weights_validation.query("Level_id == 'Level12'")
                   .rename({"Agg_Level_1":"item_id", "Agg_Level_2":"store_id", "Weight":"weight"}, axis=1)
                   .drop("Level_id", axis=1)
                  )

In [8]:
weights_level12

,item_id,store_id,weight
12350,FOODS_1_001,CA_1,1.970000e-05
12351,FOODS_1_001,CA_2,1.850000e-05
12352,FOODS_1_001,CA_3,1.430000e-05
12353,FOODS_1_001,CA_4,5.380000e-06
12354,FOODS_1_001,TX_1,5.980000e-07
...,...,...,...
42835,HOUSEHOLD_2_516,TX_2,1.270000e-05
42836,HOUSEHOLD_2_516,TX_3,7.920000e-06
42837,HOUSEHOLD_2_516,WI_1,1.580000e-06
42838,HOUSEHOLD_2_516,WI_2,1.580000e-06


In [11]:
weights_level12["item_id"] = item_encoder.transform(weights_level12.loc[:, ["item_id"]])
weights_level12["store_id"] = store_encoder.transform(weights_level12.loc[:, ["store_id"]])

In [14]:
weights_level12.to_parquet("../input/weights_level12.parquet", index=False)

***
## Weights for level 9

In [21]:
weights_validation.query("Level_id == 'Level9'").head()

,Level_id,Agg_Level_1,Agg_Level_2,Weight
84,Level9,CA_1,FOODS_1,0.005208
85,Level9,CA_1,FOODS_2,0.014867
86,Level9,CA_1,FOODS_3,0.041885
87,Level9,CA_1,HOBBIES_1,0.017208
88,Level9,CA_1,HOBBIES_2,0.000602


In [22]:
weights_level9 = (weights_validation.query("Level_id == 'Level9'")
                  .rename({"Agg_Level_1":"store_id", "Agg_Level_2":"dept_id", "Weight":"weight"}, axis=1)
                  .drop("Level_id", axis=1)
                 )

In [23]:
weights_level9.head()

,store_id,dept_id,weight
84,CA_1,FOODS_1,0.005208
85,CA_1,FOODS_2,0.014867
86,CA_1,FOODS_3,0.041885
87,CA_1,HOBBIES_1,0.017208
88,CA_1,HOBBIES_2,0.000602


In [24]:
weights_level9["store_id"] = store_encoder.transform(weights_level9.loc[:, ["store_id"]])
weights_level9["dept_id"] = dept_encoder.transform(weights_level9.loc[:, ["dept_id"]])

***